In [32]:
import matplotlib.pyplot as plt
import datetime
import pandas as pd

In [41]:
import backtrader as bt
## Backtrader is the backtesting engine we're using
## A 'behind the scenes' broker is always in place
## and a starting capital amount is set in this case as $10,000
## these are all default initial conditions

import datetime  # For datetime objects
import yfinance as yf #Importing yfinance datafeed

import backtrader.feeds as btfeeds
from datetime import datetime

# Create a Stratey
class TestStrategy(bt.SignalStrategy):
         def __init__(self):
            sma = bt.ind.SMA(period=200)
            price = self.data
            crossover = bt.ind.CrossOver(price,sma)
            self.signal_add(bt.SIGNAL_LONG, crossover)
        
# Create a cerebro entity
cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy)

# Set Starting Capital at $100,000
cerebro.broker.setcash(100000) ##Set Starting Capital at $100,000

# Create a datafeed
spyticker = yf.Ticker("SPY")
data = spyticker.history(period="max", interval="1d", start="1999-01-01", end="2023-09-01" , auto_adjust=True, rounding=True)
print(data)
cerebro.adddata(data)
cerebro.run()

##print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.plot()

##print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

                             Open    High     Low   Close     Volume  \
Date                                                                   
1999-01-04 00:00:00-05:00   79.11   80.29   78.05   78.89    9450400   
1999-01-05 00:00:00-05:00   78.83   80.07   78.83   79.79    8031000   
1999-01-06 00:00:00-05:00   80.68   81.92   80.63   81.72    7737700   
1999-01-07 00:00:00-05:00   81.04   81.58   80.66   81.32    5504900   
1999-01-08 00:00:00-05:00   82.20   82.40   80.78   81.92    6224400   
...                           ...     ...     ...     ...        ...   
2023-08-25 00:00:00-04:00  437.14  439.75  433.47  438.42  102325100   
2023-08-28 00:00:00-04:00  440.69  441.84  438.42  441.20   61595400   
2023-08-29 00:00:00-04:00  441.09  447.87  440.90  447.58   83081900   
2023-08-30 00:00:00-04:00  447.93  450.08  447.20  449.42   69053900   
2023-08-31 00:00:00-04:00  450.06  451.24  448.58  448.77   66084600   

                           Dividends  Stock Splits  Capital Gai

AttributeError: 'DataFrame' object has no attribute 'setenvironment'